<a href="https://colab.research.google.com/github/EvaKolosova/MachineLearningCourse/blob/master/PriceOfFlatsInMoscow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np              # Массивы (матрицы, векторы, линейная алгебра)
import matplotlib.pyplot as plt # Научная графика
import pandas as pd             # Таблицы и временные ряды (dataframe, series)
import seaborn as sns           # Еще больше красивой графики для визуализации данных
import sklearn                  # Алгоритмы машинного обучения

# Оценка стоимости квартиры в Москве

Предположим, что вы хотите продать квартиру и желаете узнать, по какой цене следует продавать, чтобы не продешевить. Или же из этой статистики можно узнать, по каким примерно ценам сечас продаются квартиры в разных районах Москвы.
Загрузим данные о ценах на квартиры (данные собраны ВШЭ для спецкурса более 2х лет назад):

In [7]:
url = "https://raw.githubusercontent.com/EvaKolosova/MachineLearningCourse/master/flats_moscow.csv"
data_raw = pd.read_csv(url)

Таблица содержит данные о ценах для 2040 квартир в г.Москве. Для каждой квартиры указаны ее цена (в 1000$), цена и некоторые переменные, такие как общая и жилая площадь, размер кухни, расстояние до центра (в км) и до метро (в минутах), также указан тип здания (1 – кирпичный, монолит ж/б, 0 – другой), этаж (1 – этаж кроме первого и последнего, 0 – иначе.) и др.

Известно, что в датасете представлены 11 признаков и данные можно разделить на 8 классов по территориальному различию. Также известно, что отсутствуют пропущенные значения, и классы несбалансированы.

Вначале посмотрим на размеры таблицы - количество строк (каждая строка соответствует одной квартире) и количество столбцов (каждый столбец соответствует конкретному признаку):

In [8]:
data_raw.shape

(2040, 11)

Отсюда видно, что в дтасете есть 11 признаков.
Метод `data_raw.head()` отображает первые строки (по умолчанию, 5 строк) таблицы

In [9]:
data_raw.head()

,Unnamed: 0,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
0,1,81,58,40,6.0,12.5,7,1,1,1,3
1,2,75,44,28,6.0,13.5,7,1,0,1,6
2,3,128,70,42,6.0,14.5,3,1,1,1,3
3,4,95,61,37,6.0,13.5,7,1,0,1,1
4,5,330,104,60,11.0,10.5,7,0,1,1,3


А вот последние строки:

In [10]:
data_raw.tail()

,Unnamed: 0,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
2035,2036,110,77,45,10.0,12.0,5,0,0,1,5
2036,2037,95,60,43,6.0,9.0,5,0,0,1,4
2037,2038,95,60,46,5.0,10.5,5,1,0,1,7
2038,2039,129,76,48,10.0,12.5,5,0,0,1,3
2039,2040,103,64,45,7.0,15.5,5,1,0,1,1


Мы видим, что столбцы (признаки) имеют следующие имена:

- `price` - цена (в тыс. долларов.)
- `totalsp` - общая площадь квартиры (в кв.м)
- `livesp` - жилая площадь квартиры (в кв.м)
- `kitsp` - площадь кухни (в кв.м)
- `dist` - расстояние от центра в км.
- `metrdist` - расстояние до метро в минутах
- `walk` - тип передвижения (1 – пешком от метро, 0 – на транспорте)
- `brick` - тип строения здания (1 – кирпичный, монолит ж/б, 0 – другой)
- `floor` - номер этажа (1 – этаж кроме первого и последнего, 0 – иначе)
- `code` - число от 1 до 8, при помощи которого мы группируем наблюдения по территориальным подвыборкам (это и будут наши 8 классов данных, с которыми мы будем работать): 
1. Наблюдения сгруппированы на севере, вокруг Калужско-Рижской линии метрополитена 
2. Север, вокруг Серпуховско-Тимирязевской линии метрополитена 
3. Северо-запад, вокруг Замоскворецкой линии метрополитена 
4. Северо-запад, вокруг Таганско-Краснопресненской линии метрополитена 
5. Юго-восток, вокруг Люблинской линии метрополитена 
6. Юго-восток, вокруг Таганско-Краснопресненской линии метрополитена 
7. Восток, вокруг Калиниской линии метрополитена 
8. Восток, вокруг Арбатско-Покровской линии метрополитена

Признаки `walk`, `brick`, `floor` и `code` - номинальные (категориальные), но представленные в числовом виде, остальные признаки (включая целевой признак `price`) - количественные (числовые).

Требуется предсказать категориальный признак `code`, представленный в числовом виде с номером типа по остальным признакам. Это *задача классификации*.

Имена столбцов (признаков) можно получить таким образом:

In [11]:
data_raw.columns

Index(['Unnamed: 0', 'price', 'totsp', 'livesp', 'kitsp', 'dist', 'metrdist',
       'walk', 'brick', 'floor', 'code'],
      dtype='object')

Имена строк (совпадают с номерами)

In [12]:
data_raw.index

RangeIndex(start=0, stop=2040, step=1)

### Доступ к отдельным элементам и выделение подтаблиц (slice)

  Cпособ доступа    |      Элемент  | Подтаблица
 ------------------ | ------------- | -----------
  Доступ по имени   | `at`, `loc`   | `loc`
  Доступ по номеру  | `iat`, `iloc` | `iloc`
